<a href="https://colab.research.google.com/github/NDsasuke/Gradient-decent--simplex-method--Binary-linear-programming/blob/main/Binary%20Linear%20Programming/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing necessary libraries:

This segment imports the required libraries for data manipulation (pandas and numpy), data retrieval (pandas_datareader and yfinance), and linear programming (pulp).

In [10]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf
from pulp import *


# Setting up Yahoo Finance API:
This line sets up the Yahoo Finance API for data retrieval using the pandas_datareader library.

In [11]:
# Set up Yahoo Finance API
yf.pdr_override()

# Defining the list of assets:
This list contains the symbols or tickers of the assets you want to include in the portfolio. You can modify this list to include the desired assets.



In [12]:
# Define the list of assets and their symbols
assets = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Setting the start and end dates:
These variables define the time range for data retrieval. Adjust the start and end dates according to your desired historical data period.

In [13]:
# Set the start and end dates for data retrieval
start_date = '2020-01-01'
end_date = '2021-12-31'

# Fetching historical stock prices:
This line retrieves the historical stock prices for the specified assets from Yahoo Finance using the get_data_yahoo() function. It fetches the adjusted close prices ('Adj Close') for the given assets and time range.

In [14]:
# Fetch the historical stock prices for the assets
df_prices = pdr.get_data_yahoo(assets, start=start_date, end=end_date)['Adj Close']


[*********************100%***********************]  4 of 4 completed


# Calculating returns and covariances:
These lines calculate the expected returns and covariances from the fetched stock prices. The pct_change() function computes the daily percentage change in prices, and mean() and cov() functions calculate the mean and covariance matrix, respectively. The multiplication by 252 is used to annualize the values assuming 252 trading days in a year.

In [15]:
# Calculate the returns and covariances from the prices
returns = df_prices.pct_change().mean() * 252
covariances = df_prices.pct_change().cov() * 252


# Creating the binary linear programming problem
This line creates a new linear programming problem with the name "Portfolio_Optimization" and sets it up for maximization (LpMaximize).

In [16]:
# Create the binary linear programming problem
prob = LpProblem("Portfolio_Optimization", LpMaximize)

# Defining decision variables:
This line defines the decision variables x as a dictionary of binary variables, with each variable corresponding to an asset in the assets list.

In [17]:
# Define the decision variables
x = LpVariable.dicts("asset", assets, cat="Binary")

# Defining the objective function:
This line defines the objective function to maximize the total return of the portfolio. It is the sum of the product of each asset's return (returns[i]) and its corresponding decision variable (x[i]).

In [18]:
# Define the objective function
prob += lpSum([returns[i] * x[i] for i in assets]), "Total_Return"

# Defining constraints:
These lines define the constraints for the linear programming problem. The first constraint ensures that the weights of the assets sum up to 1, representing a fully invested portfolio. The second constraint sets a minimum investment requirement of 80% of the portfolio value.

In [19]:
# Define the constraints
prob += lpSum([x[i] for i in assets]) == 1, "Sum_of_Weights"
prob += lpSum([x[i] for i in assets]) >= 0.8, "Minimum_Investment"

# Solving the problem:
This line solves the binary linear programming problem and finds the optimal solution that maximizes the objective function while satisfying the defined constraints.

In [20]:
# Solve the problem
prob.solve()

1

# Printing the optimal asset allocation:
This segment constructs a DataFrame called allocation by concatenating individual DataFrames for each asset's allocation. The value(x[i]) retrieves the optimal value of the decision variable for each asset. Finally, the allocation DataFrame is printed, displaying the assets and their corresponding allocations (0 or 1).

In [21]:
# Print the optimal asset allocation
allocation = pd.concat([pd.DataFrame({"Asset": i, "Allocation": value(x[i])}, index=[0]) for i in assets], ignore_index=True)
print(allocation)


   Asset  Allocation
0   AAPL         1.0
1  GOOGL         0.0
2   MSFT         0.0
3   AMZN         0.0
